In [1]:
#HTML과 XML 문서를 파싱하기위한 파이썬 패키지 다운로드
#!pip install beautifulsoup4 
#!pip install selenium

# selenium 패키지를 사용하기 위한 Chrome Driver 다운로드
1. Chrome 다운로드
2. 버전에 맞는 Chrome Driver 다운로드 
    * Chrome 버전 확인: 크롬 > 더보기 > 도움말 > Chrome 정보
    * Chrome Driver 다운로드: https://sites.google.com/a/chromium.org/chromedriver/downloads
3. Chrome Driver를 현재 소스코드와 동일한 디렉토리로 이동

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import quote_plus
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time
import os

In [3]:
NUM = 1000 #크롤링할 이미지 최대 개수(이미지는 NUM이하로 크롤링됨)

In [4]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(3)

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
def loadHtml(path):
    driver.get(path)
    driver.implicitly_wait(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup.find_all('img')

In [ ]:
def crawlingImage(n,savepath,failStack):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    img = soup.find_all('img')
    print(len(img), n)
    if len(img)<=n:
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        result = driver.execute_script(
            '''
            if(document.getElementsByClassName("mye4qd")[0].getAttribute("value")==="결과 더보기")
                document.getElementsByClassName("mye4qd")[0].click()
            '''
            )
        failStack += 1
        if failStack > 5:
            return n, -1
        else:
            return n, failStack
    
    for i in img[n:]:
        print(n)
        try:
            imgUrl = i['src']
        except KeyError:
            try:
                imgUrl = i['data-src']
            except KeyError:
                continue
    
        with urlopen(imgUrl) as f:
            with open(savepath + str(n)+'.jpg','wb') as h: # w - write b - binary
                img = f.read()
                h.write(img)
        with Image.open(savepath + str(n)+'.jpg') as image:
            plt.imshow(image)
            plt.show()
        n += 1
        #driver.execute_script("window.scrollTo(0, {})".format(n*(231//4)))
        
        if n > NUM:
            break
    failStack = 0
    return n, failStack

# crawling "fire" image in google

In [ ]:
img = loadHtml('https://www.google.co.kr/search?hl=ko&tbm=isch&source=hp&biw=1036&bih=674&ei=Dlw2YJLRH5fr-Qbrx7SQDg&q=fire&oq=fire&gs_lcp=CgNpbWcQAzIFCAAQsQMyAggAMgIIADICCAAyBQgAELEDMgIIADICCAAyAggAMggIABCxAxCDATICCABQ0wdYzgpg9gtoAHAAeACAAYsBiAHwA5IBAzAuNJgBAKABAaoBC2d3cy13aXotaW1n&sclient=img&ved=0ahUKEwiSkLKx14LvAhWXdd4KHesjDeIQ4dUDCAc&uact=5')

n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/1/img',failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

# Crawling ""fire smoke in city" OR "fire smoke"" image in google

In [ ]:
img = loadHtml('https://www.google.com/search?q=%22fire+smoke+in+city%22+OR+%22fire+smoke%22&tbm=isch&ved=2ahUKEwj4pIjJ15HvAhVC6pQKHf-6CK0Q2-cCegQIABAA&oq=%22fire+smoke+in+city%22+OR+%22fire+smoke%22&gs_lcp=CgNpbWcQA1CsK1iQLGD5LmgAcAB4AIABWYgBqQGSAQEymAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=hDk-YPjSDsLU0wT_9aLoCg&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870')
n = 0
failStack = 0
while n<NUM :
    n, failStack = crawlingImage(n,'./data/crawled_images/2/img', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

# Crawling "school OR factory OR school OR street OR landscape" image in google

In [ ]:
img = loadHtml('https://www.google.com/search?as_st=y&tbm=isch&hl=ko&as_q=school+OR+factory+OR+school+OR+street+OR+landscape&as_epq=&as_oq=&as_eq=&cr=&as_sitesearch=&safe=active&tbs=itp:photo,ic:color,ift:jpg')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/0/img', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

In [ ]:
cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, 'data')
crawled_dir = os.path.join(data_dir, 'crawled_images')
class0_dir = os.path.join(crawled_dir,'0')
class1_dir = os.path.join(crawled_dir,'1')
class2_dir = os.path.join(crawled_dir,'2')

normal_img_files = os.listdir(class0_dir)
fire_img_files = os.listdir(class1_dir)
smoke_img_files = os.listdir(class2_dir)

for img in normal_img_files:
    img_path = os.path.join(class0_dir,img)
    image = Image.open(img_path)
    image_mode = image.mode
    if image_mode != 'RGB':
        print(img, image_mode)
        image = np.asarray(image)
        print(image.shape)
        os.remove(img_path)

for img in fire_img_files:
    img_path = os.path.join(class1_dir,img)
    image = Image.open(img_path)
    image_mode = image.mode
    if image_mode != 'RGB':
        print(img, image_mode)
        image = np.asarray(image)
        print(image.shape)
        os.remove(img_path)
        
for img in smoke_img_files:
    img_path = os.path.join(class2_dir,img)
    image = Image.open(img_path)
    image_mode = image.mode
    if image_mode != 'RGB':
        print(img, image_mode)
        image = np.asarray(image)
        print(image.shape)
        os.remove(img_path)